# Húsnæðislánareiknivél

Þetta er lauflétt húsnæðislánareiknivél sem ég bjó til til þess að skilja betur hvernig húsnæðislán virka.

Áður en þú byrjar að nota reiknivélina þarftu að setja upp þau forritasöfn sem eru nauðsynleg. Það er gert með því að keyra `python -m pip install -r requirements.txt`. Þessi skrifta hefur verið prófuð á Python útgáfu 3.11.2.

Til að nota reiknivélina er nóg breyta breytunum í næsta hólfi (Stillingar) og keyra reiknivélina (Run all).

## Verkefnalisti:

- [ ] Yfirlit yfir tölfræði lána (t.d. heildarkostnað, lægsta, hæsta og miðgildi afborgunar)
- [x] Gröf og yfirlit yfir eignamyndun
- [ ] Núvirðing á upphæðum miðað við verðbólgu

In [ ]:
# Stillingar - Hægt að breyta þessum gildum

# Sameiginlegar stillingar fyrir öll lán
lansupphaed: int = 20000000

# Óverðtryggð lán
lanstimi_overdtryggt: int = 40
vextir_overdtryggt: float = 0.105  # Ársvextir

# Verðtryggð lán
verdbolga: float = 0.095  # Áætluð árleg
lanstimi_verdtryggt: int = 15
vextir_verdtryggt: float = 0.0265  # Ársvextir

# Fasteign
verd: int = 60000000
fasteignamat: int = 50000000

# Mánaðarleg viðbótargreiðsla inn á lán
vidbotargreidsla: int = 0

In [ ]:
# Afleiddar breytur - Ekki breyta

fjoldi_afborgana_overdtryggt = 12 * lanstimi_overdtryggt
fjoldi_afborgana_verdtryggt = 12 * lanstimi_verdtryggt

In [ ]:
# Innfluttningur á kóðasöfnum

from typing import Generator

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Reikniföll

def jafnar_afborganir(
    lansupphaed: int,
    fjoldi_afborgana: int,
    vextir: float,
    vidbotargreidsla: int,
) -> Generator[tuple[int, int, int, int, int, int, int], None, None]:
    '''
    Þetta fall reiknar út greiðslur lán með jöfnum afborgunum.

    Fallið skilar generator sem skilar við hverja ítrun færslu sem inniheldur:
        - númer mánaðar
        - greiðslu af höfuðstól
        - vaxtagreiðsla
        - viðbótargreiðsla inn á höfuðstól
        - heildar greiðsla
        - verðbætur (Alltaf 0 þar sem lánið er óverðtryggt)
        - stöðu höfuðstóls
    '''
    hofudstoll = float(lansupphaed)
    for i in range(fjoldi_afborgana):
        afborgun_af_hofudstol = hofudstoll / (fjoldi_afborgana - i + 1) + vidbotargreidsla
        vaxtagreidsla = hofudstoll * (vextir) / 12
        
        if hofudstoll - afborgun_af_hofudstol <= 0:
            afborgun_af_hofudstol = hofudstoll

        heildargreidsla = afborgun_af_hofudstol + vaxtagreidsla

        yield (
            i+1,
            int(round(afborgun_af_hofudstol, 0)),
            int(round(vaxtagreidsla, 0)),
            vidbotargreidsla,
            int(round(heildargreidsla, 0)),
            0,
            int(round(hofudstoll, 0)),
        )

        hofudstoll -= afborgun_af_hofudstol 


def jafnar_afborganir_verdtryggt(
    lansupphaed: int,
    fjoldi_afborgana: int,
    vextir: float,
    verdbolga: float,
    vidbotargreidsla: int,
) -> Generator[tuple[int, int, int, int, int, int, int], None, None]:
    '''
    Þetta fall reiknar út greiðslur lán með jöfnum afborgunum.

    Fallið skilar generator sem skilar við hverja ítrun færslu sem inniheldur:
        - númer mánaðar
        - greiðslu af höfuðstól
        - vaxtagreiðsla
        - viðbótargreiðsla inn á höfuðstól
        - heildar greiðsla
        - verðbætur
        - stöðu höfuðstóls
    '''
    # Verðbólgan er gefin upp á ársgrundvelli. Við verðum 
    # því að umreikna hana yfir á mánaðargrundvöll.
    # Við þurfum að leysa eftirfarandi jöfnu:
    #   1 + V = (1 + m)^12
    # þar sem V er árleg verðbólga og m er verðbólgan á mánaðargurndvelli.
    # Leysum hana og fáum
    #   m = (1 + V)^(1/12) - 1
    manadarleg_verdbolga = (1 + verdbolga)**(1/12) - 1

    hofudstoll = float(lansupphaed)
    for i in range(fjoldi_afborgana):
        verdbaetur =  hofudstoll * manadarleg_verdbolga
        hofudstoll += verdbaetur

        vaxtagreidsla = hofudstoll * (vextir / 12)
        afborgun_af_hofudstol = hofudstoll / (fjoldi_afborgana - i ) + vidbotargreidsla  # afborgun_af_hofudstol * (1 + ((verdbolga / 100) / 12))
        
        if hofudstoll - afborgun_af_hofudstol <= 0:
            afborgun_af_hofudstol = hofudstoll
        
        heildargreidsla = afborgun_af_hofudstol + vaxtagreidsla

        yield (
            i+1,
            int(round(afborgun_af_hofudstol, 0)),
            int(round(vaxtagreidsla, 0)),
            vidbotargreidsla,
            int(round(heildargreidsla, 0)),
            int(round(verdbaetur, 0)),
            int(round(hofudstoll, 0)),
        )

        hofudstoll -= afborgun_af_hofudstol


def jafnar_greidslur(
    lansupphaed: int,
    fjoldi_afborgana: int,
    vextir: float,
    vidbotargreidsla: int,
) -> Generator[tuple[int, int, int, int, int, int, int], None, None]:
    '''
    Þetta fall reiknar út greiðslur láns með jöfnum greiðslum.

    Greiðslur inn á höfuðstól eru reiknaðar með formúlunni:

        M = H[r(1 + r)^n]/[(1 + r)^n – 1]

    þar sem:
        - M er mánaðarleg heildargreiðsla
        - H er höfuðstóll láns
        - r er vextir á mánuði (Vextir á ársgrundvelli / 12)
        - n er fjöldi afborgana (mánuðir)

    Fallið skilar generator sem skilar við hverja ítrun færslu sem inniheldur:
        - númer mánaðar
        - greiðslu af höfuðstól
        - vaxtagreiðsla
        - viðbótargreiðsla
        - heildar greiðsla
        - verðbætur (Alltaf 0 þar sem lánið er óverðtryggt)
        - stöðu höfuðstóls
    '''
    hofudstoll = float(lansupphaed)

    r = vextir / 12

    for i in range(fjoldi_afborgana):
        vaxtagreidsla = hofudstoll * r
        heildargreidsla = hofudstoll * (r * (1 + r)**(fjoldi_afborgana - i)) / ((1 + r)**(fjoldi_afborgana - i) - 1) + vidbotargreidsla
        afborgun_af_hofudstol = heildargreidsla - vaxtagreidsla

        if hofudstoll - afborgun_af_hofudstol <= 0:
            afborgun_af_hofudstol = hofudstoll
            vidbotargreidsla = 0

        yield (
            i+1,
            int(round(afborgun_af_hofudstol, 0)),
            int(round(vaxtagreidsla, 0)),
            vidbotargreidsla,
            int(round(heildargreidsla, 0)),
            0,
            int(round(hofudstoll, 0)),
        )

        hofudstoll -= afborgun_af_hofudstol


def jafnar_greidslur_verdtryggt(
    lansupphaed: int,
    fjoldi_afborgana: int,
    vextir: float,
    verdbolga: float,
    vidbotargreidsla: int,
) -> Generator[tuple[int, int, int, int, int, int, int], None, None]:
    '''
    Þetta fall reiknar út greiðslur verðtryggðs láns með jöfnum greiðslum.

    Greiðslur inn á höfuðstól eru reiknaðar með formúlunni:

        M = H[r(1 + r)^n]/[(1 + r)^n – 1]

    þar sem:
        - M er mánaðarleg heildargreiðsla
        - H er höfuðstóll láns
        - r er vextir á mánuði (Vextir á ársgrundvelli / 12)
        - n er fjöldi afborgana (mánuðir)

    Þar sem lánið er verðtryggt þarf að endurreikna greiðsluna inn á höfuðstól
    í hverjum mánuði miðað við þann fjölda afborgana sem eftir eru af láninu.

    Fallið skilar generator sem skilar við hverja ítrun færslu sem inniheldur:
        - númer mánaðar
        - greiðslu af höfuðstól
        - vaxtagreiðsla
        - viðbótargrieðsla
        - heildar greiðsla
        - verðbætur
        - stöðu höfuðstóls
    '''
    hofudstoll = float(lansupphaed)
    r = vextir / 12

    # Verðbólgan er gefin upp á ársgrundvelli. Við verðum 
    # því að umreikna hana yfir á mánaðargrundvöll.
    # Við þurfum að leysa eftirfarandi jöfnu:
    #   1 + V = (1 + m)^12
    # þar sem V er árleg verðbólga og m er verðbólgan á mánaðargurndvelli.
    # Leysum hana og fáum
    #   m = (1 + V)^(1/12) - 1
    manadarleg_verdbolga = (1 + verdbolga)**(1/12) - 1

    for i in range(fjoldi_afborgana):
        verdbaetur =  hofudstoll * manadarleg_verdbolga
        hofudstoll += verdbaetur

        fjoldi_afborgana_eftir = fjoldi_afborgana - i
        heildargreidsla = hofudstoll * (r * (1 + r)**fjoldi_afborgana_eftir) / ((1 + r)**fjoldi_afborgana_eftir - 1) + vidbotargreidsla
        vaxtagreidsla = hofudstoll * r
        afborgun_af_hofudstol = heildargreidsla - vaxtagreidsla

        if hofudstoll - afborgun_af_hofudstol <= 0:
            afborgun_af_hofudstol = hofudstoll
            vidbotargreidsla = 0

        yield (
            i+1,
            int(round(afborgun_af_hofudstol, 0)),
            int(round(vaxtagreidsla, 0)),
            vidbotargreidsla,
            int(round(heildargreidsla, 0)),
            int(round(verdbaetur, 0)),
            int(round(hofudstoll, 0)),
        )

        hofudstoll -= afborgun_af_hofudstol

In [ ]:
# Teikni- og hjálparföll

def varpa_i_DataFrame(gogn: Generator) -> pd.DataFrame:
    '''
    Þetta fall tekur inn gögn sem skilað er frá reikningnum og varpar þeim í
    pandas.DataFrame sem hægt er að vinna með.
    '''
    return pd.DataFrame(
        gogn,
        columns=[
            'Mánuður',
            'Afborgun',
            'Vaxtagreiðsla',
            'Viðbótargreiðsla',
            'Heildargreiðsla',
            'Verðbætur',
            'Höfuðstóll',
        ],
    ).set_index('Mánuður')


def teikna_greidslur(gogn: pd.DataFrame, titill: str) -> None:
    plt.figure()
    plt.plot(gogn['Afborgun'], label='Afborgun')
    plt.plot(gogn['Vaxtagreiðsla'], label='Vaxtagreiðsla')
    plt.plot(gogn['Heildargreiðsla'], label='Heildargreiðsla')

    plt.xlabel('Mánuður')
    plt.ylabel('Upphæð (kr.)')
    plt.title(titill)
    plt.legend()


def teikna_heildaruppaedir(gogn: pd.DataFrame, titill: str) -> None:
    samtals_greitt = gogn['Heildargreiðsla'].cumsum()
    samtals_afborganir = gogn['Afborgun'].cumsum()
    
    plt.figure()
    plt.plot(samtals_greitt, label='Samtals greitt')
    plt.plot(samtals_afborganir, label='Samtals afborganir af höfuðstól')
    plt.plot(gogn['Höfuðstóll'], label='Höfuðstóll')
    plt.xlabel('Mánuður')
    plt.ticklabel_format(style='plain', axis='y', useOffset=False)
    plt.ylabel('Upphæð (kr.)')
    plt.title(titill)
    plt.legend()

def _nuvirdi(upphaed: int, timabil: int, verdbolga_timabils: float,) -> int:
    return int(round(upphaed / (1 + verdbolga_timabils)**timabil, 2))


def reikna_nuvirdi(gogn: pd.DataFrame, verdbolga: float) -> pd.DataFrame:
    nuvirt_gogn = gogn.copy() # We don't want to modify the original data

    def _nuvirda_linu(lina: pd.Series, timabil: int, verdbolga_timabils: float) -> pd.Series:
        return lina.apply(_nuvirdi, args=(timabil, verdbolga_timabils,))

    # Verðbólgan er gefin upp á ársgrundvelli. Við verðum 
    # því að umreikna hana yfir á mánaðargrundvöll.
    # Við þurfum að leysa eftirfarandi jöfnu:
    #   1 + V = (1 + m)^12
    # þar sem V er árleg verðbólga og m er verðbólgan á mánaðargurndvelli.
    # Leysum hana og fáum
    #   m = (1 + V)^(1/12) - 1
    manadarleg_verdbolga = (1 + verdbolga)**(1/12) - 1

    nuvirt_gogn = gogn.copy().apply(lambda lina: _nuvirda_linu(lina, lina.name, manadarleg_verdbolga), axis=1)

    return nuvirt_gogn


In [ ]:
# Sambanburður á verðtryggðum og óverðtryggðum lánum með jafnar afborganir og jöfnum greiðslum

jafnar_afborganir_overdtryggt = varpa_i_DataFrame(
        jafnar_afborganir(
            lansupphaed=lansupphaed,
            fjoldi_afborgana=fjoldi_afborgana_overdtryggt,
            vextir=vextir_overdtryggt,
            vidbotargreidsla=vidbotargreidsla,
        )
    )

jafnar_greidslur_overdtryggt = varpa_i_DataFrame(
        jafnar_greidslur(
            lansupphaed=lansupphaed,
            fjoldi_afborgana=fjoldi_afborgana_overdtryggt,
            vextir=vextir_overdtryggt,
            vidbotargreidsla=vidbotargreidsla,
        )
    )

jafnar_afborganir_verdtryggt = varpa_i_DataFrame(
        jafnar_afborganir_verdtryggt(
            lansupphaed=lansupphaed,
            fjoldi_afborgana=fjoldi_afborgana_verdtryggt,
            vextir=vextir_verdtryggt,
            verdbolga=verdbolga,
            vidbotargreidsla=vidbotargreidsla,
        )
    )

jafnar_greidslur_verdtryggt = varpa_i_DataFrame(
        jafnar_greidslur_verdtryggt(
            lansupphaed=lansupphaed,
            fjoldi_afborgana=fjoldi_afborgana_verdtryggt,
            vextir=vextir_verdtryggt,
            verdbolga=verdbolga,
            vidbotargreidsla=vidbotargreidsla,
        )
    )

teikna_greidslur(jafnar_afborganir_overdtryggt, 'Greiðslubyrði láns með jöfnum afborgunum (óverðtryggt)')
teikna_heildaruppaedir(jafnar_afborganir_overdtryggt, 'Heildarupphæð greidd af láni með jöfnum afborgunum (óverðtryggt)')

teikna_greidslur(jafnar_greidslur_overdtryggt, 'Greiðslubyrði láns með jöfnum greiðslum (óverðtryggt)')
teikna_heildaruppaedir(jafnar_greidslur_overdtryggt, 'Heildarupphæð greidd af láni með jöfnum greiðslum (óverðtryggt)')

teikna_greidslur(jafnar_afborganir_verdtryggt, 'Greiðslubyrði láns með jöfnum afborgunum (verðtryggt)')
teikna_heildaruppaedir(jafnar_afborganir_verdtryggt, 'Heildarupphæð greidd af láni með jöfnum afborgunum (verðtryggt)')

teikna_greidslur(jafnar_greidslur_verdtryggt, 'Greiðslubyrði láns með jöfnum greiðslum (verðtryggt)')
teikna_heildaruppaedir(jafnar_greidslur_verdtryggt, 'Heildarupphæð greidd af láni með jöfnum greiðslum (verðtryggt)')

In [ ]:
# Núvirtur sambanburður á verðtryggðum og óverðtryggðum lánum með jafnar afborganir og jöfnur greiðslum 


jafnar_afborganir_overdtryggt_nuvirt = reikna_nuvirdi(jafnar_afborganir_overdtryggt, verdbolga)
jafnar_greidslur_overdtryggt_nuvirt = reikna_nuvirdi(jafnar_greidslur_overdtryggt, verdbolga)
jafnar_afborganir_verdtryggt_nuvirt = reikna_nuvirdi(jafnar_afborganir_verdtryggt, verdbolga)
jafnar_greidslur_verdtryggt_nuvirt = reikna_nuvirdi(jafnar_greidslur_verdtryggt, verdbolga)

teikna_greidslur(jafnar_afborganir_overdtryggt_nuvirt, 'Greiðslubyrði láns með jöfnum afborgunum, núvirt (óverðtryggt)')
teikna_heildaruppaedir(jafnar_afborganir_overdtryggt_nuvirt, 'Heildarupphæð greidd af láni með jöfnum afborgunum, núvirt (óverðtryggt)')

teikna_greidslur(jafnar_afborganir_verdtryggt_nuvirt, 'Greiðslubyrði láns með jöfnum afborgunum, núvirt (verðtryggt)')
teikna_heildaruppaedir(jafnar_afborganir_verdtryggt_nuvirt, 'Heildarupphæð greidd af láni með jöfnum afborgunum, núvirt (verðtryggt)')

teikna_greidslur(jafnar_greidslur_overdtryggt_nuvirt, 'Greiðslubyrði láns með jöfnum greiðslum, núvirt (óverðtryggt)')
teikna_heildaruppaedir(jafnar_greidslur_overdtryggt_nuvirt, 'Heildarupphæð greidd af láni með jöfnum greiðslum, núvirt (óverðtryggt)')

teikna_greidslur(jafnar_greidslur_verdtryggt_nuvirt, 'Greiðslubyrði láns með jöfnum greiðslum, núvirt (verðtryggt)')
teikna_heildaruppaedir(jafnar_greidslur_verdtryggt_nuvirt, 'Heildarupphæð greidd af láni með jöfnum greiðslum, núvirt (verðtryggt)')